## Rolling sum

In [1]:
import pandas as pd
import numpy as np
import string

from numba import jit

import warnings
from numba.core.errors import NumbaTypeSafetyWarning
warnings.simplefilter("ignore", category=NumbaTypeSafetyWarning)

In this notebook I'm comparing several approaches to compute the rolling sum in a pandas dataframe.

I'm collecting the computational time and at the end we can see all the times collected inside a table.

In [2]:
WINDOW = 3

In [3]:
time_result = {}    # table for the time comparisons

In [4]:
rows = 30000
cols = 14
col_names = list(string.ascii_uppercase[:cols])

In [5]:
np.random.seed(seed=12345)
df = pd.DataFrame(np.random.randint(0, 11, size=(rows, cols)), columns=col_names)

In [6]:
df.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,2,5,1,4,9,5,2,1,6,1,9,7,6,0
1,2,9,10,1,2,6,7,7,7,8,7,1,7,4
2,0,3,5,7,3,1,5,2,5,3,8,5,2,5
3,3,0,6,8,0,5,10,6,8,9,2,2,2,9
4,10,7,5,7,1,0,10,9,10,3,0,3,0,6


#### Approach that uses numpy roll

In [7]:
def rolling_sum(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """ Rolling sum using np.roll approach """
    dt = df.values.astype(np.float64)
    dt_arr = [dt]
    for i in range(1, window):
        dt_i = np.roll(dt, shift=i, axis=0)
        dt_arr.append(dt_i)
    df_sum = np.sum(dt_arr, axis=0)
    df_sum[:window-1,:] = np.nan
    return pd.DataFrame(df_sum, columns=df.columns)

In [8]:
df_roll = rolling_sum(df, window=WINDOW)

In [9]:
df_roll.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,17.0,16.0,12.0,14.0,12.0,14.0,10.0,18.0,12.0,24.0,13.0,15.0,9.0
3,5.0,12.0,21.0,16.0,5.0,12.0,22.0,15.0,20.0,20.0,17.0,8.0,11.0,18.0
4,13.0,10.0,16.0,22.0,4.0,6.0,25.0,17.0,23.0,15.0,10.0,10.0,4.0,20.0


#### Direct approach:   O(n * window)

In [10]:
def rolling_classic(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """ Rolling sum using manual rolling """
    
    arr = df.values.astype(np.float64)
    rows, cols = arr.shape
    result = np.full((rows, cols), np.nan, dtype=np.float64)

    for i in range(window - 1, rows):
        sum_row = np.zeros(cols, dtype=np.float64)
        for j in range(window):
            sum_row += arr[i - j]  # Accumulate the sum of the window
        result[i] = sum_row

    return pd.DataFrame(result, columns=df.columns)

In [11]:
roll_class = rolling_classic(df, WINDOW)

In [12]:
roll_class.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,17.0,16.0,12.0,14.0,12.0,14.0,10.0,18.0,12.0,24.0,13.0,15.0,9.0
3,5.0,12.0,21.0,16.0,5.0,12.0,22.0,15.0,20.0,20.0,17.0,8.0,11.0,18.0
4,13.0,10.0,16.0,22.0,4.0,6.0,25.0,17.0,23.0,15.0,10.0,10.0,4.0,20.0


#### Pandas rolling function

In [13]:
roll = df.rolling(window=WINDOW).sum()

In [14]:
roll.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,17.0,16.0,12.0,14.0,12.0,14.0,10.0,18.0,12.0,24.0,13.0,15.0,9.0
3,5.0,12.0,21.0,16.0,5.0,12.0,22.0,15.0,20.0,20.0,17.0,8.0,11.0,18.0
4,13.0,10.0,16.0,22.0,4.0,6.0,25.0,17.0,23.0,15.0,10.0,10.0,4.0,20.0


### Let's time the functions:

In [15]:
r_np_roll = %timeit -o rolling_sum(df, window=WINDOW)

time_result["numpy.roll"] = r_np_roll.average * 1000

1.3 ms ± 21 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [16]:
r_class = %timeit -o rolling_classic(df, window=WINDOW)

time_result["O(n * window)"] = r_class.average * 1000

35.9 ms ± 138 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
r_cyth = %timeit -o df.rolling(window=WINDOW).sum(engine="cython")

time_result["pandas cython"] = r_cyth.average * 1000

3.99 ms ± 47.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


----------------------------------------------------------------------------

In [18]:
r_p_num = %timeit -o df.rolling(window=WINDOW).sum(engine="numba", \
            engine_kwargs={"parallel": False, "nogil": False, "nopython": True})

time_result["pandas numba"] = r_p_num.average * 1000

2.71 ms ± 108 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
r_p_par = %timeit -o df.rolling(window=WINDOW).sum(engine="numba", \
        engine_kwargs={"parallel": True, "nogil": True, "nopython": True})

time_result["pandas parallel"] = r_p_par.average * 1000

1.11 ms ± 97.6 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


### The for loops in rolling_classic are slow. Let's use numba

In [20]:
@jit(nopython=True)
def rolling_sum_numba(arr: np.ndarray, window: int) -> np.ndarray:
    """ Rolling sum using manual rolling approach compatible with Numba JIT compilation """
    
    rows, cols = arr.shape
    result = np.full((rows, cols), np.nan, dtype=np.float64)

    for i in range(window - 1, rows):
        sum_row = np.zeros(cols, dtype=np.float64)
        for j in range(window):
            sum_row += arr[i - j]  # Accumulate the sum of the window
        result[i] = sum_row

    return result

In [21]:
def rolling_classic_numba(df: pd.DataFrame, window: int) -> pd.DataFrame:
    arr = df.values.astype(np.float64)
    result_array = rolling_sum_numba(arr, window)
    result_df = pd.DataFrame(result_array, columns=df.columns)
    return result_df


In [22]:
roll_numb = rolling_classic_numba(df, window=WINDOW)

In [23]:
roll_numb.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,17.0,16.0,12.0,14.0,12.0,14.0,10.0,18.0,12.0,24.0,13.0,15.0,9.0
3,5.0,12.0,21.0,16.0,5.0,12.0,22.0,15.0,20.0,20.0,17.0,8.0,11.0,18.0
4,13.0,10.0,16.0,22.0,4.0,6.0,25.0,17.0,23.0,15.0,10.0,10.0,4.0,20.0


In [24]:
r_c_n = %timeit -o rolling_classic_numba(df, window=WINDOW)

time_result["O(n * window) numba"] = r_c_n.average * 1000

1.52 ms ± 26.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Rolling approach that is O(n)

In [25]:
def rolling_fast(arr: np.ndarray, window: int) -> np.ndarray:
    """ Rolling sum using the approach that add the first and remove the last """
    rows, cols = arr.shape
    result = np.full((rows, cols), np.nan, dtype=np.float64)  # initialize
    running_sum = np.zeros(cols, dtype=np.float64)
    for i in range(rows):
        running_sum += arr[i]                 # add the first 
        if i >= window:
            running_sum -= arr[i - window]    # remove the last
        if i >= window - 1:
            result[i] = running_sum           # store value
    return result

def rolling_sum_fast(df: pd.DataFrame, window: int) -> pd.DataFrame:
    arr = df.values.astype(np.float64)
    result_array = rolling_fast(arr, window)
    result_df = pd.DataFrame(result_array, columns=df.columns)
    return result_df

In [26]:
roll_fast = rolling_sum_fast(df, WINDOW)
roll_fast.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,17.0,16.0,12.0,14.0,12.0,14.0,10.0,18.0,12.0,24.0,13.0,15.0,9.0
3,5.0,12.0,21.0,16.0,5.0,12.0,22.0,15.0,20.0,20.0,17.0,8.0,11.0,18.0
4,13.0,10.0,16.0,22.0,4.0,6.0,25.0,17.0,23.0,15.0,10.0,10.0,4.0,20.0


In [27]:
rolling_fast_numba = jit(rolling_fast, nopython=True)

def rolling_sum_fast_numba(df: pd.DataFrame, window: int) -> pd.DataFrame:
    arr = df.values.astype(np.float64)
    result_array = rolling_fast_numba(arr, window)
    result_df = pd.DataFrame(result_array, columns=df.columns)
    return result_df

In [28]:
roll_fast = rolling_sum_fast_numba(df, WINDOW)
roll_fast.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,17.0,16.0,12.0,14.0,12.0,14.0,10.0,18.0,12.0,24.0,13.0,15.0,9.0
3,5.0,12.0,21.0,16.0,5.0,12.0,22.0,15.0,20.0,20.0,17.0,8.0,11.0,18.0
4,13.0,10.0,16.0,22.0,4.0,6.0,25.0,17.0,23.0,15.0,10.0,10.0,4.0,20.0


In [29]:
r_f_n = %timeit -o rolling_sum_fast_numba(df, window=WINDOW)

time_result["O(n) numba"] = r_f_n.average * 1000

808 μs ± 32.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [30]:
r_f = %timeit -o rolling_sum_fast(df, window=WINDOW)

time_result["O(n)"] = r_f.average * 1000

21.6 ms ± 236 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
df_time = pd.DataFrame.from_dict(time_result, orient="index", columns=["ms"]).sort_values(
    by="ms", ascending=True)

In [32]:
df_time

,ms
O(n) numba,0.807695
pandas parallel,1.111458
numpy.roll,1.298949
O(n * window) numba,1.520822
pandas numba,2.706935
pandas cython,3.991259
O(n),21.625995
O(n * window),35.852639
